 # please re-start kernel if these packages are installed for the first time

In [1]:
# ! pip install -e /user/hwang/Jupyter_notebooks/pkgs/objFuncs_REA
# ! pip install -e /user/hwang/Jupyter_notebooks/pkgs/pyBO

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import datetime
import concurrent
from epics import caget

In [3]:
from objFuncs.REA_maximize_FC import maximize_FC1058 as maximize_FC
import objFuncs
from objFuncs.util import plot_obj_history
from pyBO import pyBO

In [4]:
from pyBO import pyBO

In [5]:
objFuncs._global_machineIO._test = False
objFuncs._global_machineIO._fetch_data_time_span = 2.05
objFuncs._global_machineIO._ensure_set_timewait_after_ramp = 0.25
# objFuncs._global_machineIO.fetch_data = objFuncs.construct_machineIO._manual_fetch_data

objFuncs._global_machineIO.view()

In [6]:
budget = 60

In [8]:
ion = caget("REA_EXP:ELMT")
Q = int(caget("REA_EXP:Q"))
A = int(caget("REA_EXP:A"))
AQ = A/Q
ion = str(A)+ion+str(Q)
print(ion, 'A/Q=',AQ)

In [9]:
now0 = datetime.datetime.now()
now0str = str(now0)[:str(now0).rfind(':')].replace(' ','_').replace(':','').replace('-','')
fname = '[REA][pyBO]'+now0str+'_'+ion+'_FC_CM4in'
fname

# Decision parameters and range

In [10]:
decision_CSETs = [
    'REA_BTS46:DCH_D1529:I_CSET',
    'REA_BTS46:DCV_D1529:I_CSET',
    'REA_BTS46:DCH_D1466:I_CSET',
    'REA_BTS46:DCV_D1466:I_CSET',
                  ]

In [11]:
STC = 10.
STC1=STC
STC2=STC
STC3=STC
STC4=STC

decision_min = [-STC1*AQ,-STC2*AQ,-STC3*AQ,-STC4*AQ]
decision_max = [STC1*AQ,STC2*AQ,STC3*AQ,STC4*AQ]

#decision_min = [-50*AQ,-150*AQ,-135*AQ,-135*AQ]
#decision_max = [ 50*AQ, 150*AQ, 135*AQ, 135*AQ]
#decision_tols = None
decision_tols = [1,2,2,2]
print(decision_min)
print(decision_max)
print(decision_tols)

In [12]:
assert len(decision_CSETs) == len(decision_min) == len(decision_max)
if decision_tols is not None:
    assert len(decision_CSETs) == len(decision_tols) 
assert np.all(np.array(decision_min)<np.array(decision_max))

# Objectives

### manual definition of obj

In [13]:
FC0999 = 50e-12
#FC='REA_BTS19:MTER_N0001:I_RD'
FC='REA_BTS46:MTER_N0002:I_RD'
objective_goal   = {FC: {'more than': FC0999}}
objective_weight = {FC: 1}
objective_norm   = {FC: 10e-12}
# objective_goal   = {'REA_WK01:FC_D1058:BC_RD': {'more than': FC0999}}
# objective_weight = {'REA_WK01:FC_D1058:BC_RD': 1}
# objective_norm   = {'REA_WK01:FC_D1058:BC_RD': 1e-12}

### regularization

In [14]:
# reg_goal = {key:0.5*(val_max+val_min) if 'PSD' in key else 0 for key,val_min,val_max in zip(decision_CSETs,decision_min,decision_max)}
# reg_norm = {key:0.5*(val_max-val_min) for key,val_min,val_max in zip(decision_CSETs,decision_min,decision_max)}
# reg_weight = {key:0.1 if 'PSD' in key else 0.05 for key in decision_CSETs}   

In [15]:
# objective_goal.update(reg_goal)
# objective_norm.update(reg_norm)
# objective_weight.update(reg_weight)

In [16]:
# pd.DataFrame((objective_goal,objective_norm,objective_weight),index=['goal','norm','weight']).T

### construct objectives class

In [17]:
obj = maximize_FC(
    decision_CSETs= decision_CSETs,
    decision_min  = decision_min,
    decision_max  = decision_max,
    decision_tols = decision_tols,
    objective_goal = objective_goal,
    objective_weight = objective_weight,
    objective_norm = objective_norm,
    objective_fill_none_by_init = True,
)

 # Prepare plot callbacks

In [18]:
# define what to plot
plot_CSETs = plot_obj_history(
                obj.history['decision_CSETs'],
                title = 'decision_CSETs',
                inline = True,
                )
plot_RDs = plot_obj_history(
                obj.history['objective_RDs'],
                title = 'objective_RDs',
                inline = True,
                )
plot_objs = plot_obj_history(
            obj.history['objectives'],
            title = 'objectives',
            inline = True,
            )
callbacks = [plot_CSETs,plot_RDs,plot_objs]


# evaluate objective and plot
def obj_callbacks(x):
    return obj(x,callbacks=callbacks)

# Run BO

### initial Random search and then Global BO

In [19]:
N_random_init = int(budget/3)
N_global_iter = int(budget/3)

bo, X_pending, Y_pending_future = pyBO.runBO(
                                    obj_callbacks,  
                                    bounds=obj.decision_bounds,
                                    n_init= N_random_init,
                                    x0 = obj.x0,
                                    budget = N_random_init+N_global_iter,
                                    batch_size=1,
                                    write_log = False)
for f in callbacks:
    f.close()

### Local BO

In [20]:
# uncomment and run if more iteration needed
acquisition_func_args = {'beta':9}
N_local_iter = int(budget/3)
# N_local_iter = 10
betas = np.linspace(9,0.1,N_local_iter)

for i in range(N_local_iter):
    x0,y0 = bo.best_sofar()
    local_min = x0 - 0.05*(obj.decision_max - obj.decision_min)
    local_max = x0 + 0.05*(obj.decision_max - obj.decision_min)
    local_bounds = list(zip(local_min,local_max))
    acquisition_func_args['beta'] = betas[i]
    
    X_pending, Y_pending_future= bo.loop( 
                                    n_loop=1,  # number of additional optimization interation
                                    func_obj = obj_callbacks,
                                    X_pending = X_pending, 
                                    Y_pending_future = Y_pending_future,
                                    acquisition_func_args = acquisition_func_args,
                                    batch_size = 1,
                                    write_log = False,
                                    )
for f in callbacks:
    f.close()

In [21]:
# if python raised error, use the following to re-use all the data until now
# bo, X_pending, Y_pending_future = pyBO.runBO(
#     obj_callbacks,  
#     bounds=obj.decision_bounds,
#     n_init=n_init,
#     x0 = np.array(obj.history['decision_RDs']['values']),
#     y0 = np.array(obj.history['objectives']['values']),
#     budget = budget,
#     batch_size=1,
#     path="./log/",
#     tag=fname+'_pyBO_history',
#     write_log = False)
# for f in callbacks:
#     f.close()

In [22]:
fig,ax = plt.subplots(figsize=(4,3))
ax = bo.plot_obj_history(ax=ax,plot_best_only=True)
#ax.vlines(n_init,*ax.get_ylim(),color='k')
fig.tight_layout()
# fig.savefig(fname+'_history.png',dpi=90)

In [23]:
# set to best solution 
x_best,y_best_old = bo.best_sofar()
y_best_new = obj(x_best)
print(x_best,y_best_old[0],y_best_new)   # check if best solution objective value is consistent

In [24]:
now1 = datetime.datetime.now()
now1str = str(now1)[:str(now1).rfind(':')].replace(' ','_').replace(':','').replace('-','')
time_delta = now1 - now0
print("time took:",str(time_delta.total_seconds()/60)[:4],'min')

# Visualize Surrogate model

In [25]:
# fixed_values_for_each_dim = {2:x_best[2],3:x_best[3]}  # fix values to visualize high dim surrogate model
fixed_values_for_each_dim = None              # do not fix values but project maximum. Can take long time to plot
batch_size = 1

In [26]:
for i in range(int(len(decision_CSETs)/2)):
    fig,ax = plt.subplots(figsize=(4,3))
    bo.plot_model_2D_projection(project_maximum=True,
                                dim_xaxis = 2*i,
                                dim_yaxis = 2*i+1,
                                grid_ponits_each_dim = 16,
                                fixed_values_for_each_dim=fixed_values_for_each_dim,
                                fig=fig,ax=ax);
    ax.legend()
    pv_x = obj.decision_CSETs[2*i]
    pv_y = obj.decision_CSETs[2*i+1]
    pv_x = pv_x[pv_x.find(':')+1:]
    pv_y = pv_y[pv_y.find(':')+1:]
    pv_x = pv_x[:pv_x.find(':')].replace('_D','')
    pv_y = pv_y[:pv_y.find(':')].replace('_D','')
    ax.set_xlabel(pv_x)
    ax.set_ylabel(pv_y)
    fig.tight_layout()
#     fig.savefig(fname+'_'+pv_x+'_'+pv_y+'_.png',dpi=90)